In [2]:
import csv
import requests
import psycopg2

In [2]:
conn = psycopg2.connect(
    host="localhost",
    database="Perpustakaan_Kel-07_SBD",  
    user="postgres",
    password="feby1602",  
    port=5432
)

cursor = conn.cursor()
print("Koneksi ke database berhasil!")

Koneksi ke database berhasil!


In [3]:
def fetch_books_from_api(query="*", max_results=100):
    url = f"https://www.googleapis.com/books/v1/volumes?q={query}&maxResults=40"
    books = []
    
    while len(books) < max_results:
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            new_books = data.get("items", [])
            books.extend(new_books)
            
            next_start_index = len(books)
            if next_start_index < max_results and 'nextPageToken' in data:
                url = f"https://www.googleapis.com/books/v1/volumes?q={query}&maxResults=40&pageToken={data['nextPageToken']}"
            else:
                break
        else:
            print(f"Error fetching data: {response.status_code}")
            break

    result = []
    for book in books[:max_results]:
        volume_info = book.get("volumeInfo", {})
        title = volume_info.get("title", "Unknown")
        authors = ", ".join(volume_info.get("authors", ["Unknown"]))
        publisher = volume_info.get("publisher", "Unknown")
        published_date = volume_info.get("publishedDate", "0")[:4]
        
        result.append({
            "judul": title,
            "penulis": authors,
            "penerbit": publisher,
            "tahunTerbit": published_date if published_date.isdigit() else 0,
            "stok": 10
        })
    
    return result

In [4]:
def insert_category_if_not_exists(category_name):
    cursor.execute("SELECT id FROM kategori WHERE nama = %s", (category_name,))
    category = cursor.fetchone()

    if not category:
        cursor.execute("INSERT INTO kategori (nama) VALUES (%s) RETURNING id", (category_name,))
        conn.commit()
        category_id = cursor.fetchone()[0]
        print(f"Kategori '{category_name}' berhasil ditambahkan!")
    else:
        category_id = category[0]
        print(f"Kategori '{category_name}' sudah ada.")
    
    return category_id

In [5]:
def insert_books_to_db(books, query):
    try:
        kategori_id = insert_category_if_not_exists(query)

        for book in books:
            try:
                cursor.execute("""
                    INSERT INTO buku (judul, penulis, penerbit, tahunTerbit, stok, kategori_id)
                    VALUES (%s, %s, %s, %s, %s, %s)
                    ON CONFLICT (judul) DO NOTHING;
                """, (
                    book["judul"],
                    book["penulis"],
                    book["penerbit"],
                    int(book["tahunTerbit"]) if book["tahunTerbit"].isdigit() else 0,
                    book["stok"],
                    kategori_id
                ))
            except Exception as e:
                print(f"Error inserting book: {book['judul']}, Error: {e}")
                conn.rollback()
        conn.commit()
        print("Data berhasil dimasukkan ke database!")
    except Exception as e:
        print(f"Error in insert_books_to_db: {e}")
        conn.rollback()

In [6]:
def export_to_csv(filename, query, columns):
    cursor.execute(query)
    rows = cursor.fetchall()
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(columns)
        writer.writerows(rows)
    print(f"Data berhasil disimpan ke file {filename}!")

In [7]:
queries = [
    "art", "biography", "business", "children", "computers", "cooking", "design", 
    "history", "health", "horror", "literature", "music", "mystery", "philosophy", 
    "poetry", "psychology", "religion", "science", "self-help", "spirituality", 
    "sports", "technology", "travel", "true-crime", "fiction", "non-fiction", 
    "romance", "young-adult", "science-fiction", "fantasy"
]

for query in queries:
    books = fetch_books_from_api(query, max_results=100)
    insert_books_to_db(books, query)


Kategori 'art' sudah ada.
Data berhasil dimasukkan ke database!
Kategori 'biography' sudah ada.
Data berhasil dimasukkan ke database!
Kategori 'business' sudah ada.
Error inserting book: STARTUP BUSINESS, Error: value too long for type character varying(255)

Error inserting book: The Magic Of Thinking Big in Business Administration, Error: value too long for type character varying(255)

Error inserting book: Journal of Information Systems Engineering and Business Intelligence, Error: value too long for type character varying(255)

Error inserting book: BUSINESS COMMUNICATION: KONSEP DAN APLIKASI DALAM KONTEKS INDIVIDU, KELOMPOK, DAN ORGANISASI, Error: value too long for type character varying(255)

Data berhasil dimasukkan ke database!
Kategori 'children' sudah ada.
Data berhasil dimasukkan ke database!
Kategori 'computers' sudah ada.
Data berhasil dimasukkan ke database!
Kategori 'cooking' sudah ada.
Data berhasil dimasukkan ke database!
Kategori 'design' sudah ada.
Data berhasil dim

In [11]:
export_to_csv(
    filename="07_HasilCrawlingSistemPerpustakaanDigital_kategori.csv",
    query="SELECT id, nama FROM kategori",
    columns=["id", "nama"]
)

export_to_csv(
    filename="07_HasilCrawlingSistemPerpustakaanDigital_buku.csv",
    query="SELECT id, judul, penulis, penerbit, tahunTerbit, stok, kategori_id FROM buku",
    columns=["id", "judul", "penulis", "penerbit", "tahunTerbit", "stok", "kategori_id"]
)

InterfaceError: cursor already closed

In [1]:
cursor.close()
conn.close()
print("Koneksi ke database telah ditutup.")

NameError: name 'cursor' is not defined